In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["WANDB_SILENT"] = "true"

In [4]:
import argparse
import os
import statistics
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import yaml
from src import BertClassifier
from src import datasets as data_utils
from src import influence, train_utils, utils
from src.datasets import create_loo_dataset, create_test_sst2, create_train_sst2
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

import wandb

device = utils.get_device()

# config = utils.load_config(
#     "model_params/bert_classifier.yaml", epochs=5, num_training_examples=1000
# )

og_model, config = BertClassifier.load_model(
    "model_params/bert-epoch30-reg0.001-10000.yaml",
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Create Datasets

In [5]:
USE_BERT_EMBEDDINGS = True

# Create datasets
train_dataset = create_train_sst2(
    num_samples=config["num_training_examples"],
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
    device=device,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)

test_dataset = create_test_sst2(
    tokenizer_name=config["bert_model_name"],
    max_seq_len=config["max_sequence_length"],
    device=device,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14321.44it/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

## Train model

In [6]:
# full_model, original_df, test_loss, test_acc = train_utils.train_bert_model(
#     train_dataset=train_dataset,
#     test_dataset=test_dataset,
#     config=config,
#     use_bert_embeddings=USE_BERT_EMBEDDINGS,
# )
# test_loss, test_acc
fdf, test_loss, test_acc = train_utils.evaluate_loss(og_model, test_dataloader, use_bert_embeddings=True)
test_loss, test_acc

(0.37703973579917965, 83.14220183486239)

## Adversarial Attack

In [42]:
def perturb_datapoint(dataset, data_guid, perturbation):
    """This modifies the dataset in place"""
    device = utils.get_device()
    guid, inputs, attn_mask, labels = [t[data_guid] for t in train_dataset.tensors]
    assert guid.squeeze() == data_guid

    inputs_before = inputs.detach().clone()
    inputs += perturbation.to(device)
    return inputs_before, inputs


def perform_attack(
    model,
    config,
    train_dataset,
    test_dataset,
    target_test_guid,
    target_train_guid=None,
    alpha=2e-2,
):
    infl = None
    if target_train_guid is None:
        print("---Computing Influence Function---")
        infl = influence.compute_influence(
            model,
            target_test_guid,
            param_influence=list(model.classifier.parameters()),
            train_dataset=train_dataset,
            test_dataset=test_dataset,
            use_bert_embeddings=True,
            lissa_r=1,
            lissa_depth=1,
            damping=5e-3,
            scale=100,
        )

        # Most negative influence is most helpful
        helpful_idxs = np.argsort(infl)[:10]
        target_train_guid = helpful_idxs[0]

    print("---Computing Input Influence Function---")
    input_infl = influence.compute_input_influence(
        model,
        target_test_guid,
        param_influence=list(model.classifier.parameters()),
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        use_bert_embeddings=True,
        lissa_r=1,
        lissa_depth=1,
        damping=5e-3,
        scale=100,
        training_indices=[target_train_guid],
    )

    print(f"---Perturbing training guid {target_train_guid}---")
    perturb = alpha * input_infl[target_train_guid]
    perturb_datapoint(train_dataset, target_train_guid, perturb)

    print("---Retraining on perturbed data---")
    # Retrain model on perturbed dataset
    model, df, full_test_loss, full_test_acc = train_utils.train_bert_model(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        config=config,
        use_bert_embeddings=True,
    )
    df["perturbed_guid"] = target_train_guid
    return model, df, infl, input_infl

In [52]:
fdf[fdf.pred == fdf.label].sort_values("loss", ascending=False).iloc[200]

test_guid                        845
logits       [0.5202583, -0.6204333]
pred                               0
label                              0
loss                        0.277327
Name: 845, dtype: object

In [53]:
# Top 100 can be attacked

TEST_GUID = 845

baseline_test_loss = fdf[fdf.test_guid == TEST_GUID].loss.squeeze()
baseline_test_loss

0.27732712030410767

In [54]:
# target_train_guid = 262

hist = {
    "loss_df": [],
    "influence": [],
    "input_influence": [],
}
model = og_model
for i in range(15):
    model, loss_df, infl, input_infl = perform_attack(
        model=model,
        config=config,
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        target_test_guid=TEST_GUID,
        alpha=5e-1,
        # target_train_guid=target_train_guid,
    )
    loss_df["iter"] = i

    hist["loss_df"].append(loss_df)
    hist["influence"].append(infl)
    hist["input_influence"].append(input_infl)
    
    test_df = loss_df[loss_df.test_guid == TEST_GUID]
    if test_df.pred.squeeze() != test_df.label.squeeze():
        break

---Computing Influence Function---
LiSSA reps: 1 and num_iterations: 1000
Recursion at depth 0: norm is 7.170006
Recursion at depth 100: norm is 108.369644
Recursion at depth 200: norm is 163.181732
Recursion at depth 300: norm is 192.374741
Recursion at depth 400: norm is 208.072952
Recursion at depth 500: norm is 218.451111
Recursion at depth 600: norm is 223.446838
Recursion at depth 700: norm is 226.833298
Recursion at depth 800: norm is 229.715286
Recursion at depth 900: norm is 230.460648
Recursion at depth 999: norm is 232.348038


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 147.71it/s]


---Computing Input Influence Function---
LiSSA reps: 1 and num_iterations: 1000
Recursion at depth 0: norm is 7.114713
Recursion at depth 100: norm is 107.310768
Recursion at depth 200: norm is 163.592789
Recursion at depth 300: norm is 192.630981
Recursion at depth 400: norm is 209.661072
Recursion at depth 500: norm is 218.678299
Recursion at depth 600: norm is 223.884995
Recursion at depth 700: norm is 226.265518
Recursion at depth 800: norm is 229.755447
Recursion at depth 900: norm is 229.281372
Recursion at depth 999: norm is 230.660950


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 89.16it/s]


---Perturbing training guid 146---
---Retraining on perturbed data---


...

---Perturbing training guid 237---
---Retraining on perturbed data---


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initial 0.41442395658465236, 80.27522935779817


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 30.91batch/s]


Final 0.4088614702345444, 81.9954128440367


In [51]:
df = pd.concat(hist["loss_df"])
df[df.test_guid == TEST_GUID]

,test_guid,logits,pred,label,loss,perturbed_guid,iter
835,835,"[-0.2794927, 0.1258931]",1,1,0.510857,283,0
835,835,"[-0.23010865, 0.07639963]",1,1,0.551591,674,1
835,835,"[-0.21975267, 0.06545623]",1,1,0.560677,636,2
835,835,"[-0.22391771, 0.06937532]",1,1,0.557215,861,3
835,835,"[-0.19686149, 0.042281836]",1,1,0.580707,91,4
835,835,"[-0.18992358, 0.03470982]",1,1,0.587125,486,5
835,835,"[-0.15941639, 0.004119546]",1,1,0.614719,281,6
835,835,"[-0.1041629, -0.051154472]",1,1,0.666994,94,7
835,835,"[-0.0863075, -0.06904605]",1,1,0.684554,693,8
835,835,"[-0.028598854, -0.12658453]",0,1,0.743340,746,9


In [25]:
df = pd.concat(hist["loss_df"])
df[df.test_guid == TEST_GUID]

,test_guid,logits,pred,label,loss,perturbed_guid,iter
175,175,"[0.674652, -0.7781812]",0,1,1.663018,455,0
175,175,"[0.72075385, -0.82425386]",0,1,1.738360,861,1
175,175,"[0.745123, -0.84857845]",0,1,1.778663,693,2
175,175,"[0.7943765, -0.8980169]",0,1,1.861358,486,3
175,175,"[0.8068535, -0.90997225]",0,1,1.882031,94,4
175,175,"[0.81793123, -0.9210918]",0,1,1.900880,91,5
175,175,"[0.82862556, -0.93167734]",0,1,1.919008,636,6
175,175,"[0.8473495, -0.95056677]",0,1,1.951190,686,7
175,175,"[0.8550431, -0.9582516]",0,1,1.964397,910,8
175,175,"[0.86175734, -0.9652028]",0,1,1.976157,837,9


True

In [ ]:
-

## Scratch

In [ ]:
model2, loss_df2 = perform_attack(model, config, train_dataset, test_dataset, TEST_GUID)
loss_df2[loss_df2.test_guid == TEST_GUID]

In [ ]:
model3, loss_df3 = perform_attack(
    model2, config, train_dataset, test_dataset, TEST_GUID
)
loss_df3[loss_df3.test_guid == TEST_GUID]

In [ ]:
-

## Compute Influence Function

In [ ]:
TEST_GUID = 716

infl = influence.compute_influence(
    full_model,
    TEST_GUID,
    param_influence=list(full_model.classifier.parameters()),
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
    lissa_r=2,
    lissa_depth=1,
    damping=5e-3,
    scale=100,
)

In [ ]:
# Most negative influence is most helpful
helpful_idxs = np.argsort(infl)[:10]
helpful_idxs

In [ ]:
np.take(infl, helpful_idxs)

## Compute Input Influence Function

In [ ]:
input_infl = influence.compute_input_influence(
    full_model,
    TEST_GUID,
    param_influence=list(full_model.classifier.parameters()),
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
    lissa_r=2,
    lissa_depth=1,
    damping=5e-3,
    scale=100,
    training_indices=helpful_idxs,
)

In [ ]:
best_idx = helpful_idxs[0]
best_idx

## Perturb the Best Idx

In [ ]:
def get_guid(dataset, data_guid):
    pass


def perturb_datapoint(dataset, data_guid, perturbation):
    """This modifies the dataset in place"""
    device = utils.get_device()
    guid, inputs, attn_mask, labels = [t[data_guid] for t in train_dataset.tensors]
    assert guid.squeeze() == data_guid

    inputs_before = inputs.detach().clone()
    inputs += perturbation.to(device)
    return inputs_before, inputs

In [ ]:
alpha = 1e-2

perturb = alpha * input_infl[best_idx]
before, after = perturb_datapoint(train_dataset, best_idx, perturb)

## Retrain model with new dataset

In [ ]:
model, df, full_test_loss, full_test_acc = train_utils.train_bert_model(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    config=config,
    use_bert_embeddings=USE_BERT_EMBEDDINGS,
)

In [ ]:
test_loss = df[df.test_guid == TEST_GUID].loss.squeeze()
test_loss

In [ ]:
df[df.test_guid == TEST_GUID]

In [ ]:
--

In [ ]:
before

In [ ]:
after

In [ ]:
guid, inputs, attn_mask, labels = [t[data_guid] for t in train_dataset.tensors]

inputs[0][0]

In [ ]:
guid, inputs, attn_mask, labels = [t[262] for t in train_dataset.tensors]

In [ ]:
inputs[0][0]

In [ ]:
# inputs[0]

In [ ]:
inputs.shape

In [ ]:
inputs += perturb.to(device)

In [ ]:
inputs

In [ ]:
input_infl[262]

In [ ]:
import pickle

with open("input_infl.pkl", "wb") as fh:
    pickle.dump(input_infl, fh)

In [ ]:
262 in helpful_idxs

In [ ]:
-

## Word2Vec

In [ ]:
from gensim.models import KeyedVectors, Word2Vec

w2v = KeyedVectors.load("word2vec/glove-twitter-100.kv")

In [ ]:
w2v["spielberg"]